<a href="https://colab.research.google.com/github/azsides/QuantEcon.jl/blob/master/Firm_Example_MChen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as scp

matrix = np.array([[1, 1, 0, 0], [1, 1, 1, 0], [0, 1, 1, 0], [0, 0, 0, 1]])
print(matrix)

[[1 1 0 0]
 [1 1 1 0]
 [0 1 1 0]
 [0 0 0 1]]


Try 1: Using single linkage clustering (Note: this algorithm does not work.  Go to "Try 2")

In [ ]:
link_matrix = scp.cluster.hierarchy.linkage(matrix, metric='cityblock')
print(link_matrix)

[[0. 1. 1. 2.]
 [2. 4. 1. 3.]
 [3. 5. 3. 4.]]


Sum each column of the matrix:

In [ ]:
print(link_matrix.sum(axis=0))

[ 5. 10.  5.  9.]


We take the index of the unique values (e.g. 10, 9); the indices are 2 and 4.

Let's try a larger example?

In [ ]:
np.random.seed(1)
matrix = np.random.randint(0, 2, (4,4))
matrix ^= matrix.T
np.fill_diagonal(matrix, 1)
print(matrix)

[[1 0 1 0]
 [0 1 1 0]
 [1 1 1 0]
 [0 0 0 1]]


From manually looking at the matrix:
Unique columns are 3, 4


In [ ]:
link_matrix = scp.cluster.hierarchy.linkage(matrix, method='complete', metric='cityblock')
print(link_matrix)
print(link_matrix.sum(axis=0))

[[0. 2. 1. 2.]
 [1. 4. 2. 3.]
 [3. 5. 4. 4.]]
[ 4. 11.  7.  9.]


Hmm, so that doesn't seem to work.  Let's check the rank of the matrix


In [ ]:
print(np.linalg.matrix_rank(matrix))


4


Try 2: I think there can be a greedy approach?  Maybe this algorithm would work:

1) Sum all entries of each column

In [ ]:
col_sum = matrix.sum(axis=0)
print(col_sum)

[2 2 3 1]


2) Choose the index of the highest number.  This is your first 'firm'.  (Note: python uses 0-indexing)

In [ ]:
firm = np.argmax(col_sum)
print(firm)

2


3) "Mask out" the rows using the 1s in the selected column

In [ ]:
selected_col = matrix[:][2]
print(selected_col)
non_zero_indices = np.nonzero(selected_col)
print(non_zero_indices[0])
for i in non_zero_indices[0]:
  print(i)
  matrix[i][:] = 0
print(matrix)


[1 1 1 0]
[0 1 2]
0
1
2
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]]


4) Take the sum over all columns again, and repeat.  Here, column 4 (index 3) is the only remaining nonzero value, so we're done!

Let's manually try this for a larger 6x6 example:

In [ ]:
np.random.seed(1)
matrix = np.random.randint(0, 2, (6,6))
matrix ^= matrix.T
np.fill_diagonal(matrix, 1)
print(matrix)
print("Taking the sum over columns:")
print(matrix.sum(axis=0))
print("The last column is the largest, and we can mask everything out")
print("That leaves us with column 3 (index 2), so the answer here is firm 3 and firm 6")

[[1 0 0 0 0 1]
 [0 1 0 0 0 1]
 [0 0 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 1]
 [1 1 0 1 1 1]]
Taking the sum over columns:
[2 2 1 2 2 5]
The last column is the largest, and we can mask everything out
That leaves us with column 3 (index 2), so the answer here is firm 3 and firm 6


Let's try a different seed for the 6x6 case

In [ ]:
np.random.seed(2)
matrix = np.random.randint(0, 2, (6,6))
matrix ^= matrix.T
np.fill_diagonal(matrix, 1)
print(matrix)
print("Taking the sum:")
print(matrix.sum(axis=0))

[[1 1 0 0 1 0]
 [1 1 1 1 0 1]
 [0 1 1 1 1 1]
 [0 1 1 1 1 0]
 [1 0 1 1 1 1]
 [0 1 1 0 1 1]]
Taking the sum:
[3 5 5 4 5 4]


Yikes.  Is there a unique case here?  I don't think so.
Firms 2, 3, 5 all have 5 providers, and they're all just missing a different one.  So, you could take any combo of one of those firms + another one?